In [34]:
import pandas as pd
import numpy as np


In [35]:
files_dir = "C:\\Users\\Francesco Marrocco\\Desktop\\WPI\\SENIOR\\DS3010\\Project\\restaurant_data"

sample_size = 5000

In [ ]:
train_cust =pd.read_csv(files_dir + '\\train_customers.csv',parse_dates=['created_at','updated_at'])
test_cust =pd.read_csv(files_dir + '\\test_customers.csv',parse_dates=['created_at','updated_at'])
train_loc =pd.read_csv(files_dir + '\\train_locations.csv')
test_loc =pd.read_csv(files_dir + '\\test_locations.csv')
vendors =pd.read_csv(files_dir + '\\vendors.csv',parse_dates=['created_at','updated_at'])
orders =pd.read_csv(files_dir + '\\orders.csv',parse_dates=['created_at','delivery_date'])
submission =pd.read_csv(files_dir + '\\SampleSubmission (1).csv')

C:\Users\Francesco Marrocco\AppData\Local\Temp\ipykernel_43228\1782764812.py:6: DtypeWarning: Columns (15,16,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  orders =pd.read_csv(files_dir + '\\orders.csv',parse_dates=['created_at','delivery_date'])


In [68]:
train_cust= train_cust.rename({'akeed_customer_id': 'customer_id'}, axis=1)
test_cust= test_cust.rename({'akeed_customer_id': 'customer_id'}, axis=1)

In [ ]:
train_loc.head(2)

,customer_id,location_number,location_type,latitude,longitude
0,02SFNJH,0,NaN,1.682392,-78.789737
1,02SFNJH,1,NaN,1.679137,0.766823


In [70]:
train_cust.head()

,customer_id,created_at,updated_at
0,TCHWPBT,2018-02-07 19:16:23,2018-02-07 19:16:23
1,ZGFSYCZ,2018-02-09 12:04:42,2018-02-09 12:04:41
2,S2ALZFL,2018-03-14 18:31:43,2018-03-14 18:31:42
3,952DBJQ,2018-03-15 19:47:07,2018-03-15 19:47:07
4,1IX6FXS,2018-03-15 19:57:01,2018-03-15 19:57:01


In [71]:
test_cust=test_cust[['customer_id','created_at','updated_at']]
train_cust=train_cust[['customer_id','created_at','updated_at']];train_cust.head()

,customer_id,created_at,updated_at
0,TCHWPBT,2018-02-07 19:16:23,2018-02-07 19:16:23
1,ZGFSYCZ,2018-02-09 12:04:42,2018-02-09 12:04:41
2,S2ALZFL,2018-03-14 18:31:43,2018-03-14 18:31:42
3,952DBJQ,2018-03-15 19:47:07,2018-03-15 19:47:07
4,1IX6FXS,2018-03-15 19:57:01,2018-03-15 19:57:01


In [72]:
print(train_cust.shape,test_cust.shape,submission.shape, vendors.shape)

(34523, 3) (9753, 3) (1672000, 2) (100, 59)


### DROP DUPLICATES  CUSTOMERS

In [73]:
test_cust = test_cust.drop_duplicates(subset='customer_id', keep="first")
train_cust = train_cust.drop_duplicates(subset='customer_id', keep="first")

In [74]:
test_cust.shape

(9753, 3)

### MERGE

In [75]:
train =pd.merge(train_cust, train_loc, on ='customer_id', how ='left')
test =pd.merge(test_cust, test_loc, on ='customer_id', how ='left')

In [76]:
test.head(2)

,customer_id,created_at,updated_at,location_number,location_type,latitude,longitude
0,ICE2DJP,2018-02-07 16:45:36,2018-02-07 16:45:36,0,NaN,-96.407538,-67.197291
1,ICE2DJP,2018-02-07 16:45:36,2018-02-07 16:45:36,1,NaN,0.038654,-78.595477


In [77]:
test.shape

(16317, 7)

In [78]:
vendors.head(2)

,id,authentication_id,latitude,longitude,vendor_category_en,vendor_category_id,delivery_charge,serving_distance,is_open,OpeningTime,...,open_close_flags,vendor_tag,vendor_tag_name,one_click_vendor,country_id,city_id,created_at,updated_at,device_type,display_orders
0,4,118597.0,-0.588596,0.754434,Restaurants,2.0,0.0,6.0,1.0,11:00AM-11:30PM,...,1.0,"2,4,5,8,91,22,12,24,16,23","Arabic,Breakfast,Burgers,Desserts,Free Deliver...",Y,1.0,1.0,2018-01-30 14:42:04,2020-04-07 15:12:43,3,1
1,13,118608.0,-0.471654,0.744470,Restaurants,2.0,0.7,5.0,1.0,08:30AM-10:30PM,...,1.0,"4,41,51,34,27,15,24,16,28","Breakfast,Cakes,Crepes,Italian,Pasta,Pizzas,Sa...",Y,1.0,1.0,2018-05-03 12:32:06,2020-04-05 20:46:03,3,1


In [79]:
test=test.assign(key=1).merge(vendors.assign(key=1), on='key').drop('key',axis=1)
train=train.assign(key=1).merge(vendors.assign(key=1), on='key').drop('key',axis=1)

In [80]:
print(train.shape, test.shape, submission.shape)

(5802400, 66) (1631700, 66) (1672000, 2)


# Finding common rows (intersection) in two Pandas dataframes

In [81]:
submission.columns

Index(['CID X LOC_NUM X VENDOR', 'target'], dtype='object')

In [82]:
test['location_number_obj']=test['location_number'].astype(str)
test['id_obj']=test['id'].astype(str)
train['location_number_obj']=train['location_number'].astype(str)
train['id_obj']=train['id'].astype(str)

In [83]:
test['CID X LOC_NUM X VENDOR'] = test['customer_id'] +' X '+ test['location_number_obj'] +' X '+ test['id_obj']
train['CID X LOC_NUM X VENDOR'] = train['customer_id'] +' X '+ train['location_number_obj'] +' X '+ train['id_obj']

In [84]:
test = pd.merge(test, submission, how='outer', on=['CID X LOC_NUM X VENDOR'])

In [85]:
print(test.shape, submission.shape, train.shape)

(1672000, 70) (1672000, 2) (5802400, 69)


In [86]:
all_services = orders['CID X LOC_NUM X VENDOR'].values.tolist()
train['target']= train['CID X LOC_NUM X VENDOR'].isin(all_services).astype(int).values

In [87]:
train['target'].value_counts()

target
0    5724146
1      78254
Name: count, dtype: int64

In [88]:
test.drop('target', axis=1, inplace =True)

In [89]:
print(test.shape,train.shape)

(1672000, 69) (5802400, 70)


In [90]:
## Reduce memory usage
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [91]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

Mem. usage decreased to 2274.31 Mb (26.1% reduction)
Mem. usage decreased to 663.33 Mb (24.6% reduction)


In [93]:
train.to_csv('train.csv', index =False) 
test.to_csv('test.csv', index =False)

In [95]:
train.sample(10)

,customer_id,created_at_x,updated_at_x,location_number,location_type,latitude_x,longitude_x,id,authentication_id,latitude_y,...,country_id,city_id,created_at_y,updated_at_y,device_type,display_orders,location_number_obj,id_obj,CID X LOC_NUM X VENDOR,target
4660940,TK7Z1BT,2019-11-28 14:56:30,2019-11-28 14:56:47,2,Home,-0.046783,0.564453,188,118786.0,-0.405762,...,1.0,1.0,2019-03-03 21:04:26,2020-04-03 16:56:31,3,1,2,188,TK7Z1BT X 2 X 188,0
4719208,FAGDQ9H,2019-12-06 19:03:14,2019-12-06 19:03:20,0,NaN,-0.765137,0.219238,55,118651.0,-1.169922,...,1.0,1.0,2018-07-22 13:18:35,2020-04-02 19:49:47,3,1,0,55,FAGDQ9H X 0 X 55,0
1585470,YMH60S4,2018-12-26 21:46:25,2018-12-26 21:46:24,0,Home,0.077148,-78.625000,303,118905.0,0.005749,...,1.0,1.0,2019-07-02 14:50:31,2020-03-31 13:23:57,3,1,0,303,YMH60S4 X 0 X 303,0
4873768,NB1A8ET,2019-12-24 20:14:21,2019-12-24 20:14:46,0,Other,-0.043976,0.523438,299,118901.0,-0.818848,...,1.0,1.0,2019-06-29 13:12:01,2020-03-27 17:16:34,3,1,0,299,NB1A8ET X 0 X 299,0
273431,V8LQURR,2018-08-24 21:29:34,2018-08-24 21:29:33,2,NaN,0.020309,0.445801,148,118745.0,-0.482910,...,1.0,1.0,2018-12-30 20:20:31,2020-04-07 00:36:36,3,1,2,148,V8LQURR X 2 X 148,0
5055810,XXZ8VW2,2020-01-16 16:57:12,2020-01-16 16:57:40,0,Home,12.000000,-1.391602,67,118663.0,-0.181030,...,1.0,1.0,2018-08-09 23:41:57,2020-04-07 12:25:19,3,1,0,67,XXZ8VW2 X 0 X 67,0
3932192,PW825TQ,2019-09-21 23:31:20,2019-11-16 16:37:25,0,Other,0.657715,-78.687500,843,130447.0,-1.269531,...,1.0,1.0,2019-12-21 12:31:16,2020-04-07 19:01:17,3,1,0,843,PW825TQ X 0 X 843,0
5546313,L31M6ST,2020-02-12 15:59:47,2020-02-12 15:59:49,0,NaN,-0.031311,-0.055542,78,118675.0,-0.555176,...,1.0,1.0,2018-08-26 21:47:55,2020-03-31 22:16:15,3,1,0,78,L31M6ST X 0 X 78,0
3248294,KQ83228,2019-07-18 14:40:02,2019-10-01 18:51:01,0,Home,-1.179688,-78.437500,846,130451.0,-0.441895,...,1.0,1.0,2019-12-21 12:40:36,2020-04-06 16:19:44,3,1,0,846,KQ83228 X 0 X 846,0
1668574,GNRGEX3,2019-01-05 18:24:57,2019-01-05 18:24:57,1,Work,-0.358398,0.113220,386,118989.0,-1.001953,...,1.0,1.0,2019-08-07 15:36:33,2020-04-08 03:45:41,3,1,1,386,GNRGEX3 X 1 X 386,1
